**Data Sources**

In my analysis, I will be considering Singapore, Jakarta, Mumbai & Bangalore as key locations because of major business presence & thus driving cross-continent food habits. I will consider forsquare APIs to search key venues, trending ones, & specific italian restaurant. Some of API syntax as follows:

1) Extract all key venues in a radius parameter of 3000:
https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={}&v{}&limit=3000&radius=3000'

2) Extract trending venues, to check if our query for Italian food is one of them:
https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={}&v={}&limit=3000&radius=3000'

3) Extract all Italian venues:
https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={}&v={}&query=Italian&limit=3000&radius=3000'

4) Extract venue rating, tips related info:
https://api.foursquare.com/v2/venues/venue_id?{}?client_id={}&client_secret={}&v={}

All APIs will be used for 4 locations, & analysed using folium to check density, & then, reccomend two locations to XYZ.

In [1]:
# Please consider below step-1 for reference, other datasets also be processed similarly
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import folium
from geopy.geocoders import Nominatim 
import requests
CLIENT_ID = 'DB0N3UDWEV3FS5X4SVHT34ETE0M3E4IN0I2F25NQQKHJMIQP' # your Foursquare ID
CLIENT_SECRET = 'F5GYEZKYSPXTVZZXYRPLFGLFCSCNNQDEOOD0RLMTWFKPX4K2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DB0N3UDWEV3FS5X4SVHT34ETE0M3E4IN0I2F25NQQKHJMIQP
CLIENT_SECRET:F5GYEZKYSPXTVZZXYRPLFGLFCSCNNQDEOOD0RLMTWFKPX4K2


In [2]:
#Setting Singapore city tom extract nearby venues, & then store data in formatted dataframe
neighborhood_latitude = 1.290270 
neighborhood_longitude = 103.851959
url = 'https://api.foursquare.com/v2/venues/explore?client_id=DB0N3UDWEV3FS5X4SVHT34ETE0M3E4IN0I2F25NQQKHJMIQP&client_secret=F5GYEZKYSPXTVZZXYRPLFGLFCSCNNQDEOOD0RLMTWFKPX4K2&ll=1.290270,103.851959&v=20190404&limit=3000&radius=3000'
Singapore_json = requests.get(url).json()
venues = Singapore_json['response']['groups'][0]['items']
df = json_normalize(venues)
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
df_new =df.loc[:, filtered_columns]
df_new = df_new[['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']]

In [3]:
#Function will extract name from venue.categories
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [4]:
# Processed dataframe to finaly use in folium & plot nearby venues
df_new['venue.categories'] = df_new.apply(get_category_type, axis=1)
df_new.columns = ['name', 'categories', 'lat','lng']
df_new.columns = [col.split(".")[-1] for col in df_new.columns]
df_new.head(3)

,name,categories,lat,lng
0,National Gal­lery Singa­pore,Art Gallery,1.290740,103.851548
1,Esplanade Park,Park,1.288968,103.853580
2,Esplanade Theatre,Theater,1.289932,103.855037
